## OCR (PaddleOCR) -> OpenAI (GPT-4o-mini)

OCRを使って文字が記載されている場所にBoxを付与した画像（paddleocr_openai_extract_OCR.ipynbで作成）を、GPT-4o-miniに投げる

In [2]:
import os
from dotenv import load_dotenv
import base64
import re
import json

import pandas as pd

import openai
from openai import OpenAI

In [4]:
!ls results

menu1 menu2 menu3 menu4


In [7]:
!ls results/menu1 results/menu2 results/menu3 results/menu4

results/menu1:
menu1_ocr.jpg   replyOAI_VM.csv

results/menu2:
menu2_ocr.jpg   replyOAI_VM.csv

results/menu3:
menu3_ocr.jpg   replyOAI_VM.csv

results/menu4:
menu4_ocr.jpg   replyOAI_VM.csv


In [10]:
images = ['results/menu1/menu1_ocr.jpg', 'results/menu2/menu2_ocr.jpg', 'results/menu3/menu3_ocr.jpg', 'results/menu4/menu4_ocr.jpg']
images

['results/menu1/menu1_ocr.jpg',
 'results/menu2/menu2_ocr.jpg',
 'results/menu3/menu3_ocr.jpg',
 'results/menu4/menu4_ocr.jpg']

In [9]:
load_dotenv('.env')

client = OpenAI(api_key=os.getenv('openai-key-20241027'))
client

In [11]:
%%time

responses = {}

vision_model = "gpt-4o-mini"

prompt = '''
メニュー画像が入力されます。記載されているメニュー情報全てを抽出するのが目的です。「メニュー名」「内容説明」「価格」を以下のjson形式で出力してください。記載がない部分は「null」と出力してください。

[{
"メニュー名": "'唐揚げ'等のメニューの名前",
"内容説明": "メニュー内容の説明。記載がなければ'null'とする",
"価格": "メニューに対応した価格",
}]
'''

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

for img in images:
    print('>>>>> ', img)
    
    # Getting the base64 string
    base64_image = encode_image(img)
    #print(type(base64_image), len(base64_image))

    response = client.chat.completions.create(
        model=vision_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )
    responses[img] = response.choices[0].message.content
    #print(responses[img])

>>>>>  results/menu1/menu1_ocr.jpg
>>>>>  results/menu2/menu2_ocr.jpg
>>>>>  results/menu3/menu3_ocr.jpg
>>>>>  results/menu4/menu4_ocr.jpg
CPU times: user 82.6 ms, sys: 28 ms, total: 111 ms
Wall time: 46.8 s


In [14]:
responses.keys()

dict_keys(['results/menu1/menu1_ocr.jpg', 'results/menu2/menu2_ocr.jpg', 'results/menu3/menu3_ocr.jpg', 'results/menu4/menu4_ocr.jpg'])

In [12]:
for k,v in responses.items():
    print('>>>>> {}'.format(k))
    print(v)

>>>>> results/menu1/menu1_ocr.jpg
```json
[
    {
        "メニュー名": "ダイナソー1ポンドステーキ",
        "内容説明": "ボンド(450g)のステーキ",
        "価格": "3769 円"
    },
    {
        "メニュー名": "ダイナソー1カットステーキ",
        "内容説明": "ボンド(225g)の1ポンドステーキ",
        "価格": "2213 円"
    },
    {
        "メニュー名": "ダイナソー1ポンドハンバーグ",
        "内容説明": "ボンド(450g)の特製ハンバーグ",
        "価格": "2504 円"
    },
    {
        "メニュー名": "ダイナソー-BBQテリヤキ",
        "内容説明": "スペシャルアトロ口にはタロにすって焼き上げたし",
        "価格": "3769 円"
    },
    {
        "メニュー名": "本日の付け合わせ肉",
        "内容説明": "その他お肉に合わせたお料理",
        "価格": "7564 円"
    },
    {
        "メニュー名": "テンダーロインステーキ",
        "内容説明": "柔らかく旨味のある肉",
        "価格": "3769 円"
    }
]
```
>>>>> results/menu2/menu2_ocr.jpg
```json
[
    {
        "メニュー名": "カルボナーラ",
        "内容説明": null,
        "価格": "902"
    },
    {
        "メニュー名": "たらこスパゲッティ",
        "内容説明": null,
        "価格": "935"
    },
    {
        "メニュー名": "めんたいスパゲッティ",
        "内容説明": null,
        "価格": "935"
    },
    {
        "メニュー名": "

In [18]:
responses['results/menu4/menu4_ocr.jpg']

'```json\n[\n    {\n        "メニュー名": "焼き芋と蒸しレインボーのパーマカルチャーサラダ",\n        "内容説明": "野菜の色彩と食感にこだわったサラダ",\n        "価格": "null"\n    },\n    {\n        "メニュー名": "ローストチキンのチキンサラダ",\n        "内容説明": "白ワインのドレッシングで仕上げたワンプレートのサラダ",\n        "価格": "null"\n    },\n    {\n        "メニュー名": "きのこたちのクリームパスタ",\n        "内容説明": "クリーミーなソースと野菜の組み合わせが楽しめるパスタ",\n        "価格": "null"\n    },\n    {\n        "メニュー名": "有機ミニサラダセット",\n        "内容説明": "サラダの盛り合わせ。オーガニック野菜使用",\n        "価格": "¥1851"\n    },\n    {\n        "メニュー名": "Soup",\n        "内容説明": "有機野菜の濃厚ポタージュ",\n        "価格": "¥401"\n    },\n    {\n        "メニュー名": "キッズプレート",\n        "内容説明": "子供向けのボリューム満点プレート",\n        "価格": "¥651"\n    },\n    {\n        "メニュー名": "自家製ひしおのメニュー唐揚風",\n        "内容説明": "国産鶏肉を使った特製の唐揚げ",\n        "価格": "null"\n    },\n    {\n        "メニュー名": "豆とトマトのVEGANカレー",\n        "内容説明": "豆とトマトを使用したヘルシーなカレー",\n        "価格": "null"\n    },\n    {\n        "メニュー名": "大豆ミート唐揚げ油淋鶏風",\n        "内容説明": "サクッと揚がった大豆ミートを使用した唐揚げ",\n        "価格": "

In [22]:
responses2 = {}
dfs = {}
for k,v in responses.items():
    print('>>>>> {}'.format(k))

    # 不要文字を削除。Jason以外の一番外[]の「```json」とか
    cleaned_str = re.sub(r'^```json\n|\n```$', '', v)
    responses2[k] = cleaned_str

    # Pandasへ変換
    menu_data = json.loads(responses2[k])
    dfs[k] = pd.DataFrame(menu_data)
    print(dfs[k].shape)


>>>>> results/menu1/menu1_ocr.jpg
(6, 3)
>>>>> results/menu2/menu2_ocr.jpg
(21, 3)
>>>>> results/menu3/menu3_ocr.jpg
(10, 3)
>>>>> results/menu4/menu4_ocr.jpg
(9, 3)


In [20]:
for k,v in dfs.items():
    print('>>>>> ', k)
    display(v)

>>>>>  results/menu1/menu1_ocr.jpg


,メニュー名,内容説明,価格
0,ダイナソー1ポンドステーキ,ボンド(450g)のステーキ,3769 円
1,ダイナソー1カットステーキ,ボンド(225g)の1ポンドステーキ,2213 円
2,ダイナソー1ポンドハンバーグ,ボンド(450g)の特製ハンバーグ,2504 円
3,ダイナソー-BBQテリヤキ,スペシャルアトロ口にはタロにすって焼き上げたし,3769 円
4,本日の付け合わせ肉,その他お肉に合わせたお料理,7564 円
5,テンダーロインステーキ,柔らかく旨味のある肉,3769 円


>>>>>  results/menu2/menu2_ocr.jpg


,メニュー名,内容説明,価格
0,カルボナーラ,None,902
1,たらこスパゲッティ,None,935
2,めんたいスパゲッティ,None,935
3,スパゲティミートソース（ペペロンチーノ）,None,990
4,キノコとツナのペペロンチーノ,None,880
5,エビマカロニグラタン,None,1080
6,五目あんかけ焼きそば,None,792
7,豚肉細切りあんかけ焼きそば,None,792
8,チャーハン,None,660
9,ザーサイとレタスのチャーハン,None,748


>>>>>  results/menu3/menu3_ocr.jpg


,メニュー名,内容説明,価格
0,オムライス,ketchup-seasoned fried rice,¥900
1,大盛オムライス,ketchup-seasoned fried rice with a filling of,¥1300
2,エビフライ,fried prawns,¥1100
3,ポークカツ,pork cutlet,¥1100
4,ビーフカレー,beef curry,¥900
5,たこやき,Japanese takoyaki/octopus balls,¥400
6,ハンバーグ定食,beef hamburg steak,¥1100
7,ラーメンセット,noodle with dipping sauce,¥800
8,スパゲティ,None,¥900
9,ミートソース,None,¥500


>>>>>  results/menu4/menu4_ocr.jpg


,メニュー名,内容説明,価格
0,焼き芋と蒸しレインボーのパーマカルチャーサラダ,野菜の色彩と食感にこだわったサラダ,null
1,ローストチキンのチキンサラダ,白ワインのドレッシングで仕上げたワンプレートのサラダ,null
2,きのこたちのクリームパスタ,クリーミーなソースと野菜の組み合わせが楽しめるパスタ,null
3,有機ミニサラダセット,サラダの盛り合わせ。オーガニック野菜使用,¥1851
4,Soup,有機野菜の濃厚ポタージュ,¥401
5,キッズプレート,子供向けのボリューム満点プレート,¥651
6,自家製ひしおのメニュー唐揚風,国産鶏肉を使った特製の唐揚げ,null
7,豆とトマトのVEGANカレー,豆とトマトを使用したヘルシーなカレー,null
8,大豆ミート唐揚げ油淋鶏風,サクッと揚がった大豆ミートを使用した唐揚げ,null


In [23]:
save_dirs = ['./results/menu1/', './results/menu2', './results/menu3', './results/menu4']
save_dirs_files = [os.path.join(d, 'replyOAI_VM_withOCR.csv') for d in save_dirs]

i = 0
for k,v in dfs.items():
    print('>>>>> ', k)
    v.to_csv(save_dirs_files[i], index=False)
    i += 1

>>>>>  results/menu1/menu1_ocr.jpg
>>>>>  results/menu2/menu2_ocr.jpg
>>>>>  results/menu3/menu3_ocr.jpg
>>>>>  results/menu4/menu4_ocr.jpg
